<a href="https://colab.research.google.com/github/AbhirKarande/OCRandProductRecognition/blob/main/RFClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!git clone https://github.com/AbhirKarande/OCRandProductRecognition.git

fatal: destination path 'OCRandProductRecognition' already exists and is not an empty directory.


In [66]:
import shutil

# Specify the path of the directory you want to download
directory_path = "/content/OCRandProductRecognition/WholeFoodsTrainingImages/train"

# Create a zip file of the directory
shutil.make_archive("/content/whole_foods_training_images", "zip", directory_path)

# Download the zip file
from google.colab import files
files.download("/content/whole_foods_training_images.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
!mkdir basil

mkdir: cannot create directory ‘basil’: File exists


In [40]:
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [47]:
# Define the path to your training images
training_path = "/content/OCRandProductRecognition/WholeFoodsTrainingImages/train"

# Initialize empty lists for images and labels
images = []
labels = []

In [48]:
print(labels)

[]


In [49]:
for subdir in os.listdir(training_path):
    subdir_path = os.path.join(training_path, subdir)
    if os.path.isdir(subdir_path):
        # Read and preprocess images within each subdirectory
        for filename in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, filename)
            if os.path.isfile(file_path):
                image = cv2.imread(file_path)
                print(file_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, (100, 100))  # Resize images if needed
                images.append(image)
                labels.append(subdir)

/content/OCRandProductRecognition/WholeFoodsTrainingImages/train/s1/2.jpg
/content/OCRandProductRecognition/WholeFoodsTrainingImages/train/s1/6.jpg
/content/OCRandProductRecognition/WholeFoodsTrainingImages/train/s1/7.jpg
/content/OCRandProductRecognition/WholeFoodsTrainingImages/train/s1/1.jpg
/content/OCRandProductRecognition/WholeFoodsTrainingImages/train/s1/3.jpg
/content/OCRandProductRecognition/WholeFoodsTrainingImages/train/s1/12.jpg
/content/OCRandProductRecognition/WholeFoodsTrainingImages/train/s1/10.jpg
/content/OCRandProductRecognition/WholeFoodsTrainingImages/train/s1/5.jpg
/content/OCRandProductRecognition/WholeFoodsTrainingImages/train/s1/9.jpg
/content/OCRandProductRecognition/WholeFoodsTrainingImages/train/s1/4.jpg
/content/OCRandProductRecognition/WholeFoodsTrainingImages/train/s1/11.jpg
/content/OCRandProductRecognition/WholeFoodsTrainingImages/train/s1/8.jpg
/content/OCRandProductRecognition/WholeFoodsTrainingImages/train/s1/13.jpg
/content/OCRandProductRecognition/

In [50]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)


In [51]:
# Flatten the image arrays
images = images.reshape(images.shape[0], -1)


In [52]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)


In [53]:
# Create and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=200)
rf_classifier.fit(images, labels)


RandomForestClassifier(n_estimators=200)

In [54]:
# Predict the labels for the test set
y_pred = rf_classifier.predict(X_test)

In [55]:
print(images)

[[247 247 247 ... 247 247 247]
 [155 151 156 ... 198 188 185]
 [160 162 166 ... 182 178 172]
 ...
 [190 189 184 ...  31  69  86]
 [177 182 192 ... 107  53 101]
 [ 53  57  37 ... 159 157 154]]


In [56]:
print(y_pred)

['s2' 's3' 's3' 's1' 's3' 's1']


In [57]:
# Calculate and print the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [58]:
print(y_test)

['s2' 's3' 's3' 's1' 's3' 's1']


In [59]:
print(y_pred)

['s2' 's3' 's3' 's1' 's3' 's1']


In [60]:
import joblib
model_filename = "rfmodelnew2.pkl"
joblib.dump(rf_classifier, model_filename)

['rfmodelnew2.pkl']

In [61]:
import joblib
model_filename = "/content/rfmodelnew2.pkl"

loaded_model = joblib.load(model_filename)

In [6]:
!pip install pyheif

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 43.5 MB/s eta 0:00:00


In [26]:
import os
from PIL import Image
import pyheif


def convert_heic_to_jpg(heic_file, output_folder):
    # Load the HEIC image
    heif_file = pyheif.read(heic_file)
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
    )

    # Generate the output file path
    filename = os.path.splitext(os.path.basename(heic_file))[0]
    output_file = os.path.join(output_folder, f"{filename}.jpg")

    # Save the image as JPG
    image.save(output_file, "JPEG")


def batch_convert_heic_to_jpg(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get a list of HEIC files in the input folder
    heic_files = [
        file for file in os.listdir(input_folder)
        if file.lower().endswith(".heic")
    ]

    # Convert each HEIC file to JPG
    for heic_file in heic_files:
        heic_path = os.path.join(input_folder, heic_file)
        convert_heic_to_jpg(heic_path, output_folder)


# Specify the input and output folders
input_folder = "/content/basil"
output_folder = "/content/basil_test_images"

# Convert HEIC files to JPG
batch_convert_heic_to_jpg(input_folder, output_folder)


In [65]:
import cv2
import numpy as np
import joblib

# Load the saved model from the file
loaded_model = joblib.load(model_filename)

# Preprocess the test image
test_image_path = "/content/test_images/IMG_3864.jpg"
test_image = cv2.imread(test_image_path)
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
test_image = cv2.resize(test_image, (100, 100))  # Resize if needed
test_image = test_image.reshape(1, -1)  # Flatten the image array

# Make predictions on the test image
predicted_class = loaded_model.predict(test_image)

# Print the predicted class
print("Predicted class:", predicted_class[0])

Predicted class: s3


In [1]:
!pip install modzy-sdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [2]:

from modzy import ApiClient
from modzy._util import file_to_bytes
client = ApiClient(base_url="https://tenant18.bot.5goil.tmobile.com/api", api_key="wZr4nkIAtLFPNUPySOdC")
sources = {}
#Add any number of inputs
sources["my-input"] = {
    "input.txt": "Lorem ipsum dolor sit amet",
}
#Once you are ready, submit the job
job = client.jobs.submit_text("ed542963de", "1.0.1", sources)
print(f"job: {job}")



ERROR:modzy.http:unable to make network request
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/lib/python3.10/socket.py", line 955, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 714, in urlopen
    httplib_response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 403, in _make_request
    self._validate_conn(conn)
  File "/usr/local/lib/pyt

ValueError: ignored